In [60]:
# Import Libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [61]:
# Constants
# Data Location
LOC = "D:\DSCI\Limited_set"
Model_LOC = "alex_rip_off.h5"
# Classes to use
classes = {

}

In [62]:
def pre_process_placeholder(a):
    """
    A simple placeholder function to adjust for the lack of a need for special image pre-processing for our implementation
    This function exists to allow for simple dictionary replacement in a global-usage testing method
    :param a: the data
    :return: the data as is
    """
    return a

In [63]:
def fetch_class(res):
    """
    Function for finding the class as provided by our AlexNet implementation
    :param res: the result as produced by our AlexNet's implementation's predict method
    :return: the class identified as the most likely
    """
    return  ["n01532829", "n02802426", "n03337140", "n04579145", "n07873807"][np.argmax(res)]

In [64]:
# Dictionaries that map pretrained models to their required functionality to be subbed in during operations
models = {
    "Xception": tf.keras.applications.xception.Xception(weights="imagenet"),
    "VGG16": tf.keras.applications.vgg16.VGG16(weights="imagenet"),
    "VGG19": tf.keras.applications.vgg19.VGG19(weights="imagenet"),
    "InceptionV3": tf.keras.applications.InceptionV3(weights="imagenet"),
    "ResNet50": tf.keras.applications.resnet50.ResNet50(weights="imagenet"),
    "AlexNet": tf.keras.models.load_model(Model_LOC)
}

# Does the model allow for top 5 validation?
extended = {
    "Xception": True,
    "VGG16": True,
    "VGG19": True,
    "InceptionV3": True,
    "ResNet50": True,
    "AlexNet": False
}

# PreProcessing System
preproc = {
    "Xception": tf.keras.applications.xception.preprocess_input,
    "VGG16": tf.keras.applications.vgg16.preprocess_input,
    "VGG19": tf.keras.applications.vgg19.preprocess_input,
    "InceptionV3": tf.keras.applications.inception_v3.preprocess_input,
    "ResNet50": tf.keras.applications.resnet50.preprocess_input,
    "AlexNet": pre_process_placeholder
}

# The required decode method
decode = {
    "Xception": tf.keras.applications.imagenet_utils.decode_predictions,
    "VGG16": tf.keras.applications.imagenet_utils.decode_predictions,
    "VGG19": tf.keras.applications.imagenet_utils.decode_predictions,
    "InceptionV3": tf.keras.applications.imagenet_utils.decode_predictions,
    "ResNet50": tf.keras.applications.imagenet_utils.decode_predictions,
    "AlexNet": fetch_class
}

In [65]:
def test_model(model):
    """
    Function to test a given model and to calculate the accuracy of the model (given a pre-trained model based on a larger set, it also calculates a top 5 accuracy)
    :param model: the string model name to map through the various dictionary
    :return: the top 1 accuracy of hte model
    """
    print("Running tests on model:", model)
    count = 0
    top1 = 0
    top5 = 0
    for i in train:
        for j in range(len(i[1])):
            count += 1
            a = i[0][j]
            a = np.expand_dims(a, axis=0)
            b = np.copy(a)
            c = preproc[model](b)
            # if extended[model]:
            res = models[model].predict(c, verbose=0)
            # else:
               # res = models[model].predict(c)
            label = decode[model](res)
            top5_list = []
            if extended[model]:
                for k in label[0]:
                    top5_list.append(k[0])
                if label[0][0][0] == train.class_names[int(i[1][j])]:
                    top1 += 1
                    top5 += 1
                elif train.class_names[int(i[1][j])] in top5_list:
                    top5 += 1
            else:
                if label == train.class_names[int(i[1][j])]:
                    top1 += 1
    print("Accuracy: " + str((top1/count) * 100) + "%" )
    if extended[model]:
        print("Top 5 Accuracy: " + str((top5/count) * 100) + "%")
    print("\n\n")
    return (top1/count) * 100
    # for j in test:
    #     res = models[model].predict(j[0])



In [66]:
accuracy_dict = {} # Dictionary that can be used for populating charts or other visualizations
for model1 in models:
    shapes = models[model1].get_config()["layers"][0]["config"]["batch_input_shape"]
    shape = (shapes[1], shapes[2])
    # train, test = tf.keras.preprocessing.image_dataset_from_directory(LOC, validation_split=.99, subset="both", seed=42, image_size=shape, batch_size=32) # IF you need to split for different sizes of data
    train = tf.keras.preprocessing.image_dataset_from_directory(LOC, image_size=shape, batch_size=32)
    accuracy_dict[model1] = test_model(model1)



Found 6255 files belonging to 5 classes.
Running tests on model: Xception


KeyboardInterrupt: 